*This is pre-release documentation, but please [log issues](https://github.com/InstituteforDiseaseModeling/jupyter-notebooks-comps/issues) found.*

In [ ]:
# LOAD THE CSS!
from IPython.display import HTML
HTML("<style>{}</style>".format(open("custom_binder_styles.css").read()))

<a id="top"></a>
# COMPS website and Jupyter `v0.2.2`
---

Demonstrations of transactions between **Jupyter Notebooks** and IDM's **Computational Modeling Platform Service** (COMPS).

COMPS provides for the submission, execution, and management of computational simulations on high-performance computing clusters (HPC). COMPS employs a scaleable RESTful service architecture, including a internet accessible website, [comps.idmod.org](https://comps.idmod.org).

**There are at least two ways to interface with COMPS from a Notebook:**
* The JavaScript method of interaction with the COMPS [website](http://comps.idmod.org) is addressed here in this Notebook.
* The Python method of interaction with COMPS via the [PyCOMPS](https://github.com/InstituteforDiseaseModeling/pyCOMPS) library is addressed in [another Notebook](hello_comps_binder.ipynb).

---
### The JavaScript method
These interactions can be made from a Jupyter Notebook running in its customary browser-based read-eval-print loop environment (REPL). Since the COMPS UI is browser-based too, there is some degree of interaction that is possible between them, and this is best achieved using JavaScript.

1. [JavaScript basics](#javascript_1) in a Notebook.
2. [Open, focus, and access](#javascript_2) COMPS in another tab.
3. [Using PostMessage](#javascript_3) for communicating with COMPS.
4. [Authenticating](#javascript_4) to gain access to COMPS data.
5. [Exchanging complex data](#javascript_5) between a Notebook and COMPS.
6. [Integrating Python and JavaScript](#javascript_6) within a Notebook.
7. [Visualizing COMPS data](#javascript_7) from a Notebook.


<blockquote style="background:khaki;border-left:2px solid red;margin:0;padding:1rem"><b>NOTE:</b> Do NOT Run-All. Browser interactions can have unpredictable duration. <br/><b>NOTE:</b> This Notebook requires v3.7.0 of COMPS (yet to be released).</blockquote>

---
<a id="javascript_1" href="#top" style="float:right">TOP</a>
### 1. JavaScript basics in Notebook

1a. **The following commands** will employ Jupyter's built-in [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to run the cell block of Javascript code...

In [ ]:
%%javascript

// fundamentals...
let message = "Hello world!";
element.text(message);
alert(message);

---
<a id="javascript_2" href="#top" style="float:right">TOP</a>
### 2. Open, focus, and access COMPS in another tab 

2a. **JavaScript is the language of browsers** and will be the means to communicate between this window (where this Jupyter Notebook is running) and the COMPS website (which will exist in another tab). Start by opening a new window, then return here to this window to execute the other commands that interact with opened window in other ways. Note that a reference to the window is kept so the other commands can be addressed to it...

In [ ]:
%%javascript

// OPEN an instance of COMPS in a new tab...
window["comps_instance"] = window.open("https://comps-dev.idmod.org", "_blank");

In [ ]:
%%javascript

// GOTO to the COMPS tab...
window.comps_instance.focus();

In [ ]:
%%javascript

// CLOSE the COMPS tab...
window.comps_instance.close();
element.text("The COMPS window has been closed!")

2b. **Any window that opens another window** has a special relationship with it, but this is severely limited by browser security. Otherwise, one window could open any other website and then monkey with it. However, if these two websites are in agreement, some amount of primitive communication is possible. This time, more care is taken when opening the new window (so new windows aren't opened when these cells are rerun)...

In [ ]:
%%javascript

// STORE the address to be opened (edit as necessary)...
let CONFIG = { endpoint:"https://comps-dev.idmod.org" };

if ("comps_instance" in window && window.comps_instance != null && !window.comps_instance.closed) {
    
    // a window is already open...
    element.text("A COMPS window is already open!");

} else {
    
    // a window is not open, so open it...
    window["comps_instance"] = window.open(CONFIG.endpoint, "_blank");
    element.text("A new COMPS window has been opened!");
    
    setTimeout(function() {
        window.comps_instance.blur();
        window.focus();
    }, 1000);
}

---
<a id="javascript_3" href="#top" style="float:right">TOP</a>
### 3. Using PostMessage to communicate with COMPS

3a. **The window.postMessage method** will be the means of communicating between the browser windows. [This methodology](https://developer.mozilla.org/en-US/docs/Web/API/Window/postMessage) requires each website to be listening for communications from another. The COMPS website is already configured to listen, but this window needs a listener too. However, in the Notebook REPL, these could accumulate, so only one will be added...

In [ ]:
%%javascript

// THIS CODE IS THE ESSENTIAL LISTENER AND HANDLER FOR POSTMESSAGES

// HANDLER called upon getting a postMessage from COMPS... 
let responder = function(payload) {
    
    if ("info" in payload && "callback" in payload.info) {
        
        // a callback has been configured...
        console.log("A postMessage is being redirected to a callback!", payload);
        if (payload.info.callback in window && window[payload.info.callback] instanceof Function) {
            
            window[payload.info.callback](payload);
        }
        
    } else if ("current_element" in window && "response" in payload) {
        
        // or a current output is available...
        if (/object/i.test(typeof payload.response)) {
            window.current_element.text(`Message received: See the browser's console!`);
            console.log("A postMessage has been received!", payload); 
        } else {
            window.current_element.text(`Message received: ${payload.response}`);
        }
    
    } else {
        
        // or trace the response to the browser console...
        console.log("A postMessage has been received!", payload);
        alert(`A postMessage has been received!\nCheck the console for more!`);
    }
};

// LISTENER for postMessages from elsewhere (COMPS)...
if ("comps_listener" in window) {
    element.text("A postMessage event listener was already attached!");
} else {
    window["comps_listener"] =
    window.addEventListener("message", (event) => {
        if (!!event && "isTrusted" in event && !!event.isTrusted) {       
            if ("data" in event) {
                responder(event.data);
            }
        }
    }, true);
    element.text("A postMessage event listener has been attached!");
}

---
3b. **Sending a communication to COMPS** is expected to have a certain structure. Basically, it's an object of name:value pairs. Some of these properties are required (e/g `getter` and `observer`) telling COMPS what should be done, and others can be arbitrary or provide arguments or data required to perform the requested task. Here's a very simple example that asks the COMPS instance for its `applicationName`...

In [ ]:
%%javascript

// SEND the default response to this cell...
window["current_element"] = element;

// ASK the COMPS instance a question...
window.comps_instance.postMessage({
    getter:"comps.app.getApplicationName",
    observer:window.location.href,
    myVariable:"someValue"
}, "*");

---
3c. **Customized callbacks** will be needed according to the variety of things that can be done. Any arbitrary property (e/g `callback`) will be returned by COMPS and then can be used to disseminate the response...

In [ ]:
%%javascript

window["doThis"] = function(payload) {    
    element.text(`Message received by doThis: ${payload.response}`);
};

// ASK the COMPS instance another question...
window.comps_instance.postMessage({
    getter:"comps.app.getVersion",
    observer:window.location.href,
    callback:"doThis"
}, "*");

---
<a id="javascript_4" href="#top" style="float:right">TOP</a>
### 4. Authenticating to gain access to COMPS data

4a. **Sign-in to COMPS** can be accomplished via this same postMessage methodology, but extreme care must be taken to **never store credentials** in a Notebook. Either Python or JavaScript can be used, but whichever, a callback handler will be required to confirm success or inform of failure. Run this cell to establish the handler, then try one of the two sign-in options (JavaScript, Python) that follow...

In [ ]:
%%javascript

// THIS CODE HANDLES THE SIGNIN RESPONSE FROM COMPS

let attempts = 7
window["comps_signin"] = function(payload) {
    if (!attempts) {
        window.current_element.text(`Authentication?: failed.`);
        return;
    } else if (!!payload && "response" in payload && !!payload.response) {
        console.log("Authenticated?", payload);
        window.current_element.text(`Authentication?: ${payload.response}`);
    } else {
        --attempts;
        setTimeout(function(){
            window.comps_instance.postMessage({
                getter:"comps.auth.getToken",
                observer:window.location.href,
                callback:"comps_signin"
            }, "*");            
        }, 1000); // ALLOW TIME for signin roundtrip
    }
};
element.text("A sign-in response handler has been attached!");

---
4b. **Sign-in to COMPS using JavaScript** can be accomplished by capturing the user inputs via the browser-based `confirm` dialog and then submitting these values to COMPS...

In [ ]:
%%javascript

// SEND the default response to this cell...
window["current_element"] = element;

// CAPTURE the user's inputs...
let pw, un=prompt("COMPS UserName:"); 
if (un) {
    if (/^[a-zA-Z][a-zA-Z0-9-_\.]{1,}$/.test(un)) {
        pw = prompt("COMPS Password:");
        if (pw) {
            if (/^[^\x22]{5,256}$/.test(pw)) {
                window.comps_instance.postMessage({
                    method:"comps.auth.signin",
                    args:[{username:un,password:pw}],
                    observer:window.location.href,
                    callback:"comps_signin"
                }, "*");
                un=pw=null;
            } else {
                if (confirm("Sorry, that Password wasn't valid.")) {
                    Jupyter.notebook.execute_cells([24]);
                } 
            }
        } else {
            alert("a Password is required!");
        } 
    } else {
        if (confirm("Sorry, that UserName wasn't valid.")) {
            Jupyter.notebook.execute_cells([24]);
        } 
    }
} else {
    if (confirm("A UserName is required!")) {
        Jupyter.notebook.execute_cells([24]);
    }    
}

---
4c. **Sign-in to COMPS using Python** will prompt for credentials saved in variables that are then dispatched to COMPS as a postMessage. Care should be taken with this alternative to not retain these credentials in the Notebook's outputs. Once the COMPS instance is authenticaed, the credentials are no longer required, so they should be deleted...

In [ ]:
import re
username = input()
if re.fullmatch("[a-zA-Z][a-zA-Z0-9-_\.]{1,}", username):
    print("This username is valid. Thanks!")
else:
    print("This username is NOT valid! Try again.")

In [ ]:
import re
import getpass
password = getpass.getpass()
if re.fullmatch("[^\x22]{5,256}", password):
    print("This password is valid. Thanks!")
else:
    print("This password is NOT valid! Try again.")

In [ ]:
%%javascript

// SEND the default response to this cell...
window["current_element"] = element;

// CONVERT the Python inputs from previous cells...
IPython.notebook.kernel.execute(
    "{'username':username,'password':password}", 
    {
        iopub: {
            output: function(response) {
                let raw = response.content.data["text/plain"];
                let creds = JSON.parse(raw.replaceAll(/'/g,'"'));               
                window.comps_instance.postMessage({
                    method:"comps.auth.signin",
                    args:[creds],
                    observer:window.location.href,
                    callback:"comps_signin"
                }, "*");
            }
        }
    },
    {
        silent: false, 
        store_history: false, 
        stop_on_error: true
    }
);

---
<a id="javascript_5" href="#top" style="float:right">TOP</a>
### 5. Exchanging complex data between a Notebook and COMPS

5a. **Complex data** could require many independent calls to the COMPS database, but the COMPS website makes many of these calls itself to aggregate information for display. In some cases, it might be more convenient to simply navigate to a particular view and then capture everything that the website has assembled. This simply navigates COMPS to the latest Simulations...

In [ ]:
%%javascript
// RUN THIS CELL ONLY IF THE CELLS ABOVE HAVE NOT YET BEEN RUN!
// You will need to allow COMPS to load fully and then return to this tab.
Jupyter.notebook.execute_cells([1,14,16,24]);

In [ ]:
%%javascript

// SECOND, this handler is called upon postMessage response...
window["comps_navigation"] = function(payload) {
    if ("response" in payload) {
        
        // THIRD, go to the current user's Simulations...
        window.comps_instance.postMessage({
            method:"comps.router.locationHash",
            args:`#explore/Simulations?filters=Owner=${payload.response}`,
            observer:window.location.href
        }, "*");
        
        element.text(`COMPS has navigated to the Simulations owned by ${payload.response}`);
    } else {
        element.text("A problem has occurred.");
    }
};

// FIRST, get the current user...
window.comps_instance.postMessage({
    method:"comps.auth.getUserName",
    observer:window.location.href,
    callback:"comps_navigation"
}, "*");

---
5b. **The COMPS website is a visualization** with a wealth of data drawing many interactive layers of information. This next call gets data that has been filtered and assembled from several requests, but is now available at once...

In [ ]:
%%javascript

// FIRST, select the topmost simulation...
window.comps_instance.postMessage({
    method:"comps.modules.explore.setSelectionInSequence",
    args:1,
    observer:window.location.href
}, "*");

In [ ]:
%%javascript

// THIRD, the default response to this cell...
window["current_element"] = element;

// SECOND, get the current Simulation's data...
window.comps_instance.postMessage({
    method:"comps.modules.explore.getSelection",
    observer:window.location.href
}, "*");

---
<a id="javascript_6" href="#top" style="float:right">TOP</a>
### 6. Integrating Python and JavaScript

6a. **Storing COMPS data to Python** is achieved by executing the `IPython.notebook.kernel` upon the postMessage response, as demonstrated here...

In [ ]:
%%javascript

// STORE this Simulation Id for future steps...
window["comps_simId"] = "42a97bfe-d1c6-ec11-92e9-f0921c167864";

// STORE a Notebook variable with the response...
window["comps_getSelection"] = function(payload) {
    if ("response" in payload && "Id" in payload.response) {
        
        IPython.notebook.kernel.execute(
          "comps_json=" + JSON.stringify(JSON.stringify(payload.response, null, 4))
        );
           
        window.comps_simId = payload.response.Id;
        element.text(`Information received for Simulation Id: ${window.comps_simId}`);
        
    } else {
        element.text("No information was found!");
    }
}

// SEND the current simulation to the comps_getSelection callback...
window.comps_instance.postMessage({
    method:"comps.modules.explore.getSelection",
    observer:window.location.href,
    callback:"comps_getSelection"
}, "*");

In [ ]:
import json;

# RESULT from kernal.execute above...
print("comps_json is",type(comps_json))

# DECODE the str to a python dict...
notebook_json = json.loads(comps_json)

# CONVERT for a pretty json printout...
notebook_json_pretty = json.dumps(notebook_json, indent=4)

# PROOF...
print("notebook_json is",type(notebook_json))
print("notebook_json.DateCreated =",notebook_json["DateCreated"])
print(notebook_json_pretty)

---

6b **Fetching REST data through COMPS** does not require the website to navigate to a particular state, but is achieved by calling the COMPS API directly via the website's JavaScript library. For example, if we have a Simulation Id (from above), the `stdout.txt` can be gotten this way...

In [ ]:
%%javascript

// NOTE: this step requires window.comps_simId from previous steps!

// STORE a Notebook variable with the response...
window["comps_getOutput"] = function(payload) {
    if ("response" in payload && !!payload.response) {
        
        let output = JSON.parse(payload.response);
        let stdout = output.Resources.find(element => element.FriendlyName == "stdout.txt");
        
        IPython.notebook.kernel.execute(
          "comps_stdout_url='" + stdout.Url +"'"
        );
        
        element.text(`comps_stdout_url: ${stdout.Url}`);
        
    } else {
        element.text("No information was found!");
    }
}

// SEND the current simulation to the comps_simulation callback...
window.comps_instance.postMessage({
    rest:"comps.restclient.get",
    args:`/asset/Simulations/${window.comps_simId}/output/?flatten=1&format=json`,
    observer:window.location.href,
    callback:"comps_getOutput"
}, "*");

In [ ]:
import requests
response = requests.get(comps_stdout_url)
print(response.text)
# DISPLAY the stdout.txt...

---
<a id="javascript_7" href="#top" style="float:right">TOP</a>
### 7. Visualizing COMPS data from a Notebook

7a **To visualize the output of a COMPS Simulation** involves streaming its output data. For this example, the standard `InsetChart.json` format will be used, but this process can be adapted as required. A first step could be to get the available channels of data...

In [ ]:
%%javascript
// RUN THIS CELL ONLY IF THE CELLS ABOVE HAVE NOT YET BEEN RUN!
// You will need to allow COMPS to load fully and then return to this tab.
Jupyter.notebook.execute_cells([1,14,16,24]);

In [ ]:
%%javascript

// NOTE: COMPS Simulation (comps-dev) MUST have InsetChart.json data...
let simId = "42a97bfe-d1c6-ec11-92e9-f0921c167864";

// STORE a Notebook variable with the response...
window["comps_getChannels"] = function(payload) {
    if ("response" in payload && !!payload.response) {
        
        let info = JSON.parse(payload.response);
        console.warn("Successful REST call for Inset channels!");
        console.log(info);
        element.text(`Inset channels received: ${info.Channels}`);
        
        // STORE the channels to a python string...
        IPython.notebook.kernel.execute(
          "comps_channels_list=" + JSON.stringify(info.Channels.toString())
        );

        
    } else {
        element.text("No information was found!");
    }
}

// SEND the current simulation to the comps_getSelection callback...
window.comps_instance.postMessage({
    rest:"comps.restclient.postAsync",
    args: 
    [
        "/api/ChannelData/Metadata?format=json",
        JSON.stringify({"SimulationIds":[simId],"Filename":"output/InsetChart.json"})
    ],
    observer:window.location.href,
    callback:"comps_getChannels"
}, "*");

In [ ]:
# PRINT the channels list from python...
comps_channels_array = comps_channels_list.split(',')
print('\n'.join(comps_channels_array))

7b **To download the data** requires a POST to the COMPS REST API with a payload including the Simulation Id, the output file that holds the data (e/g InsetChart.json), and then the name of the channel desired...

In [ ]:
%%javascript

// NOTE: COMPS Simulation (comps-dev) MUST have InsetChart.json data...
let simId = "42a97bfe-d1c6-ec11-92e9-f0921c167864";
let channel = "Births";

// STORE a Notebook variable with the response...
window["comps_getData"] = function(payload) {
    if ("response" in payload && !!payload.response) {
        
        let data = JSON.parse(payload.response);
        console.warn("Successful REST call for Inset channel data!");
        element.html(`Input channel data received:<ul><li>${data.Info.join("</li><li>")}</ul>`);   
        
        // STORE the channels to a python string...
        IPython.notebook.kernel.execute(
          "comps_channel_data='" + data.Simulations[simId][channel].Data.toString() + "'"
        );

        
    } else {
        element.text("No information was found!");
    }
}

// SEND the current simulation to the comps_getSelection callback...
window.comps_instance.postMessage({
    rest:"comps.restclient.postAsync",
    args: 
    [
        "/api/ChannelData/Datastream?format=json",
        JSON.stringify({
            "SimulationIds":[simId],
            "Filename":"output/InsetChart.json",
            "Channels":[channel]
        })
    ],
    observer:window.location.href,
    callback:"comps_getData"
}, "*");

7c **To plot the data** requires converting the datastream to the proper format (int), then loading a charting library (Highcharts, as used in COMPS), and then configuring the chart for display... 

In [ ]:
# CONVERT the channel data into Python...
comps_channel_points = comps_channel_data.split(',')
points = list(map(int, comps_channel_points))
print(points)

In [ ]:
# INSTALL the highcharts library...
# !pip install python-highcharts

from highcharts import Highchart

# CONFIGURE and plot the chart...
H = Highchart(width=600, height=400)
H.set_options('title',{'text': 'Births'})
H.set_options('subtitle',{'text':'42a97bfe-d1c6-ec11-92e9-f0921c167864'})
H.set_options('xAxis',{'title':{'text':'Time Steps'}})
H.set_options('yAxis',{'title':{'text':'Births'},'lineWidth':2})
H.set_options('chart',{'backgroundColor':'transparent','style':{'fontFamily':'IBM Plex Sans'}})
H.set_options('legend',{'enabled':False})
H.set_options('tooltip',{'pointFormat':'<b>Time Step:</b> {point.x}<br/><b>Births:</b> {point.y}'})
H.add_data_set(points)
H

---
<a href="#python_1" style="float:right">TOP</a>
### Python interaction with COMPS via pyCOMPS

The Python method of interaction with COMPS via the [PyCOMPS](https://github.com/InstituteforDiseaseModeling/pyCOMPS) library is addressed in [another Notebook found here](hello_pycomps_binder.ipynb).